<center>
<table style="border:none">
    <tr style="border:none">
    <th style="border:none">
        <a  href='https://colab.research.google.com/github/AmirMardan/ml_course/blob/main/7_fully_connected_nn/0_intro_to_tensorflow.ipynb'><img src='https://colab.research.google.com/assets/colab-badge.svg'></a>
    </th>
    <th style="border:none">
        <a  href='https://github1s.com/AmirMardan/ml_course/blob/main/7_fully_connected_nn/0_intro_to_tensorflow.ipynb'><img src='../imgs/open_vscode.svg' height=20px width=115px></a>
    </th>
    </tr>
</table>
</center>


This notebook is created by <a href='https://amirmardan.github.io/'> Amir Mardan</a>. For any feedback or suggestion, please contact me via <a href="mailto:mardan.amir.h@gmail.com">email</a>, (mardan.amir.h@gmail.com).



<a name='top'></a>
# TensorFlow

TensorFlow is a powerful open-source and end-to-end platform for building machine learning models and numerical modeling.
Being end to end, we can perform all required steps of machine learning using TensorFlow.
[Here](https://www.tensorflow.org/about/case-studies) are some projects that have been done using this package. 

This notebook will cover the following topics:

- [1. Graph and Session](#graph)
    - [1.1 Build and Perform a Graph](#build_graph)
    - [1.2 Gradient in TensorFlow](#gradient)
- [2. Tensor types in TensorFlow](#types)
    - [2.1 Constant](#constant)
    - [2.2 Variable](#variable)
- [3. Tensor Manipulation](#tensor_manipulation)
    - [3.1 Creating A Tensors](#create_array)
    - [3.2 Creating Special Tensors](#special_array)
    - [3.3 Shape Manipulation](#shape)
    - [3.4 Slicing](#slicing)
- [4. Operators](#Operations)
    - [4.1 Basic Arithmetic Operators](#Arithmetic)
    - [4.2 Comparison Operators](#Comparison)
    - [4.3 Logical And Bitwise Operators](#Logical)
    

In [1]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.6.0


I'm currently using `TensorFlow version: 2.8.0`.

<a id='graph'></a>
## 1. Graph and Session



TensorFlow is a way of representing computation without performing it until it's required. 
It's simple, we first define a <em>graph</em> in Python and we perform it when it's needed.

- **step 1:** building a graph to represent the data flow.
- **step 2:** running a <em>session</em> to execute the graph.

<center>
<img src='./img/graph_session.gif' height=400px>
<br>
<b>Figure 1</b> A sample computational graph in TensorFlow (TensorFlow website).
</center>

You should know that when you want to run a graph, you're not obliged to run the whole graph. 
There is significant flexibility that you can just run a part of a graph.

All said, we should know that the explicitly defining of the graph and the session is removed for TensorFlow version 2.
So, we use `tf.function` to create a graph, and then we can execute it like normal functions in Python. 
However, there are interesting differences that we see later.


<a id='build_graph'></a>
### 1.1 Build and Perform a Graph

For the first example, let's create the following graph.

<center>
<img src='./img/graph1.png' height=200px>
</center>

This graph is actually
\begin{equation}
f(x,y) = x^2  y + y +2
\end{equation}

In [2]:
@tf.function
def f(x, y):
    x_multi = tf.multiply(x, x, name='x_multiply')
    x2y_multi = tf.multiply(x_multi, y, name='x2y_multiply')
    y2_addition = tf.add(y, 2, name='y2_addition')
    f = tf.add(x2y_multi, y2_addition)
    return f

In [3]:
x = tf.Variable(3.0)
y = tf.Variable(5.0)

result = f(x, y)
result

2022-04-24 20:02:52.908089: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-24 20:02:52.968600: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


<tf.Tensor: shape=(), dtype=float32, numpy=52.0>

As you can see, the output of this function is a ***Tensor***. 
We can convert a Tensor to a NumPy array in two ways.

In [4]:
result.numpy()

52.0

In [5]:
import numpy as np

np.array(result)

array(52., dtype=float32)

<a id='gradient'></a>
### 1.2 Gradient in TensorFlow

The simplest definition for the gradient is that the gradient is the slope of a function in different directions.
Calculating the gradient of a function is the core of training in machine learning or of any optimization problem.

The gradient of the used equation with respect to y is

\begin{equation}
\frac{\partial f(x,y)}{\partial y} = x^2 + 1,
\end{equation}
and TensorFlow allows us to compute it easily.


In [6]:
x = tf.Variable(3.0)
y = tf.Variable(1.0)

with tf.GradientTape() as g:
    g.watch(y)
    result = f(x, y)
    dy = g.gradient(result, y)
    
print(dy.numpy())

10.0


Same way, we can calculate the gradient of function with respect to both variables.

\begin{equation}
\begin{aligned}
& \frac{\partial f(x,y)}{\partial y} = x^2 + 1\\
& \frac{\partial f(x,y)}{\partial x} = 2xy
\end{aligned}
\end{equation}

In [7]:
x = tf.Variable(2.)
y = tf.Variable(3.0)

with tf.GradientTape() as g:
    g.watch([y, x])
    result = f(x, y)
    dx, dy = g.gradient(result, [x, y])
    
print(f"df/dx: {dx.numpy()}, df/dy: {dy.numpy()}")

df/dx: 12.0, df/dy: 5.0


<a id='types'></a>
## 2. Tensor types in TensorFlow

As you might have noticed, we defined the variables in the previous section using
`tf.Variable()`.
Hence, YES, we need to define the data we use with a structure that TensorFlow works with.
 


<a id='constant'></a>
### 2.1 Constant

As the name states, **Constant** is used for a constant value.
Constants create a node that takes value and it's immutable. 

```Python
tf.constant(
    value, dtype=None, shape=None, name='Const'
)
```

In [8]:
A = tf.constant([1], name='first_constant')

A

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>

We can also define multi-dimension tensors.

In [9]:
VECTOR = tf.constant([-1, 2, 3.9], name='vector')

VECTOR

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-1. ,  2. ,  3.9], dtype=float32)>

In [10]:
TWO_D = tf.constant([[-1, 2, 3.9],
                     [3, -1.2, 0]], name='2d')

TWO_D

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-1. ,  2. ,  3.9],
       [ 3. , -1.2,  0. ]], dtype=float32)>

In [11]:
THREE_D = tf.constant([[[-1, 2, 3.9],
                     [3, -1.2, 0]],
                     [[0, 1, -0.2],
                     [9, 3.2, 10]]], name='3d')

THREE_D

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[-1. ,  2. ,  3.9],
        [ 3. , -1.2,  0. ]],

       [[ 0. ,  1. , -0.2],
        [ 9. ,  3.2, 10. ]]], dtype=float32)>

<a id='variable'></a>
### 2.2 Variable

As mentioned, the value of `constant`s doesn't change. 
So, for trainable variables, we need to use another type of data which is `Variable`.


In [12]:
x = tf.Variable([[[-1, 2, 3.9],
                     [3, -1.2, 0]],
                     [[0, 1, -0.2],
                     [9, 3.2, 10]]], name='3d')

x

<tf.Variable '3d:0' shape=(2, 2, 3) dtype=float32, numpy=
array([[[-1. ,  2. ,  3.9],
        [ 3. , -1.2,  0. ]],

       [[ 0. ,  1. , -0.2],
        [ 9. ,  3.2, 10. ]]], dtype=float32)>

In [13]:
with tf.GradientTape() as gtape:
    y = x ** 2 + 10
    
gtape.gradient(y, x)

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[-2. ,  4. ,  7.8],
        [ 6. , -2.4,  0. ]],

       [[ 0. ,  2. , -0.4],
        [18. ,  6.4, 20. ]]], dtype=float32)>

<a id='tensor_manipulation'></a>
## 3. Tensor Manipulation

<a id='create_array'></a>
### 3.1 Creating A Tensor

In the previous part, we learned two types of tensors, but in this part, we learn how to build a tensor.

From the scratch, we can build a tensor using `tf.convert_to_tensor()`.

In [14]:
tf.convert_to_tensor([1, 3, -4])

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 1,  3, -4], dtype=int32)>

In [15]:
tf.convert_to_tensor([[1, 3, -4],
                      [1, 3, -4]])

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 1,  3, -4],
       [ 1,  3, -4]], dtype=int32)>

We can also convert NumPy arrays to a Tensor.

In [16]:
a = np.random.random((2, 3))

tf.convert_to_tensor(a)

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[0.36455596, 0.42898108, 0.95702029],
       [0.21399316, 0.6813637 , 0.84888731]])>

<a id='special_array'></a>
### 3.2 Creating Special Tensors

Like NumPy and special arrays, TensorFlow allows us to create some special tensors more easily.

In [17]:
# Create a 1's tensor 

tf.ones(shape=(2,3))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [18]:
# Create a 0's tensor 

tf.zeros(shape=(2,3))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

In [19]:
# Create a identity tensor 

tf.eye(num_rows=2, num_columns=3)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [0., 1., 0.]], dtype=float32)>

In [20]:
# Create a random tensor with uniform distribution

tf.random.uniform((2, 3))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.18388605, 0.6193502 , 0.73401904],
       [0.9556105 , 0.02219713, 0.8945124 ]], dtype=float32)>

In [21]:
# Create a random tensor with normal distribution

tf.random.normal((2, 3))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 2.0382524 , -0.87904227, -0.26950473],
       [ 0.16172941,  0.06466151,  0.380084  ]], dtype=float32)>

For repeatability, we can first create a seed for random values.

In [22]:
random_tensor = tf.random.Generator.from_seed(2)


rand_tensor = random_tensor.normal(shape=[2,3])
print(rand_tensor)

tf.Tensor(
[[-0.1012345  -0.2744976   1.4204658 ]
 [ 1.2609464  -0.43640924 -1.9633987 ]], shape=(2, 3), dtype=float32)


We can also shuffle a tensor.

In [23]:
b4_shuffle = random_tensor.normal(shape=(4,5))
b4_shuffle


<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[ 0.2776104 , -0.52194345,  0.1151574 ,  1.3032064 ,  1.328346  ],
       [-1.7339838 ,  0.90426886,  0.56855273, -1.2169818 ,  1.3774033 ],
       [-0.4654596 , -0.51368004, -0.5356597 , -0.4981093 , -1.2551429 ],
       [ 0.14776349, -1.0173497 , -1.0449705 ,  0.2018814 , -0.3598549 ]],
      dtype=float32)>

In [24]:
shuffled = tf.random.shuffle(b4_shuffle)
shuffled

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[ 0.14776349, -1.0173497 , -1.0449705 ,  0.2018814 , -0.3598549 ],
       [-1.7339838 ,  0.90426886,  0.56855273, -1.2169818 ,  1.3774033 ],
       [-0.4654596 , -0.51368004, -0.5356597 , -0.4981093 , -1.2551429 ],
       [ 0.2776104 , -0.52194345,  0.1151574 ,  1.3032064 ,  1.328346  ]],
      dtype=float32)>

<a id='shape'></a>
### 3.3 Shape Manipulation

We can get information about the shape and number of dimensions of a tensor as follows.

In [25]:
shuffled.shape

TensorShape([4, 5])

In [26]:
shuffled.ndim

2

We can also reshape a tensor using `tf.reshape(tensor, new_shape)`.

In [27]:
tf.reshape(shuffled, [2, 10])

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.14776349, -1.0173497 , -1.0449705 ,  0.2018814 , -0.3598549 ,
        -1.7339838 ,  0.90426886,  0.56855273, -1.2169818 ,  1.3774033 ],
       [-0.4654596 , -0.51368004, -0.5356597 , -0.4981093 , -1.2551429 ,
         0.2776104 , -0.52194345,  0.1151574 ,  1.3032064 ,  1.328346  ]],
      dtype=float32)>

Same way, we can flatten a tensor as follows

In [28]:
print(tf.reshape(shuffled, [-1]).shape)

(20,)


or presenting a tensor in form of a vector.

In [29]:
print(tf.reshape(shuffled, [-1, 1]).shape)

(20, 1)


<a id='slicing'></a>
### 3.4 Slicing

In [30]:
shuffled

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[ 0.14776349, -1.0173497 , -1.0449705 ,  0.2018814 , -0.3598549 ],
       [-1.7339838 ,  0.90426886,  0.56855273, -1.2169818 ,  1.3774033 ],
       [-0.4654596 , -0.51368004, -0.5356597 , -0.4981093 , -1.2551429 ],
       [ 0.2776104 , -0.52194345,  0.1151574 ,  1.3032064 ,  1.328346  ]],
      dtype=float32)>

<hr>
<div>
<span style="color:#151D3B; font-weight:bold">Question: 🤔</span><p>
Get the value in 2<sup>nd</sup> row and 3<sup>rd</sup> column of <code>shuffled</code>.
</div>
<hr>

In [31]:
# Answer


We can get a column as

In [32]:
shuffled[:, 2].numpy()

array([-1.0449705 ,  0.56855273, -0.5356597 ,  0.1151574 ], dtype=float32)

Same for a row

In [33]:
shuffled[2, :].numpy()

array([-0.4654596 , -0.51368004, -0.5356597 , -0.4981093 , -1.2551429 ],
      dtype=float32)

As you guess, it works exactly the same way as NumPy.

In [34]:
# Get a portion of tensor

shuffled[:2, 2:].numpy()

array([[-1.0449705 ,  0.2018814 , -0.3598549 ],
       [ 0.56855273, -1.2169818 ,  1.3774033 ]], dtype=float32)

In [35]:
# Get the last column 

shuffled[:, -1].numpy()

array([-0.3598549,  1.3774033, -1.2551429,  1.328346 ], dtype=float32)

<a id='Operations'></a>
## 4. Operators

As we created tensors, we need to be able to work with these tensors using mathematical operations.

<a id='Arithmetic'></a>
### 4.1 Basic Arithmetic Operators

In [36]:
# Let's fake up some tensors

random_seed = tf.random.Generator.from_seed(2)
tf.random.set_seed(3)

a = tf.random.normal((3, 2))
b = 3 + 2 * tf.random.normal((3, 2))
c = 1 - 3 * tf.random.normal((2, 3))

print('Shape of tensors')
print(f'a: {a.shape}')
print(f'b: {b.shape}')
print(f'c: {c.shape}')

Shape of tensors
a: (3, 2)
b: (3, 2)
c: (2, 3)


**Addition**

We can perform the addition at least two ways.

In [37]:
a + b

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[ 5.502992  ,  2.5837674 ],
       [ 6.855619  ,  5.845226  ],
       [-0.21574509,  3.5247617 ]], dtype=float32)>

In [38]:
tf.add(a, b)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[ 5.502992  ,  2.5837674 ],
       [ 6.855619  ,  5.845226  ],
       [-0.21574509,  3.5247617 ]], dtype=float32)>

**Subtraction**


In [39]:
a - b

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-7.1622663, -6.237022 ],
       [-6.605232 , -3.4912095],
       [-2.12359  , -3.5434108]], dtype=float32)>

In [40]:
tf.subtract(a, b)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-7.1622663, -6.237022 ],
       [-6.605232 , -3.4912095],
       [-2.12359  , -3.5434108]], dtype=float32)>

**Multiplication**


In [41]:
a * b 

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-5.253785  , -8.056147  ],
       [ 0.8426061 ,  5.49453   ],
       [-1.1157722 , -0.03295358]], dtype=float32)>

In [42]:
tf.multiply(a, b)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-5.253785  , -8.056147  ],
       [ 0.8426061 ,  5.49453   ],
       [-1.1157722 , -0.03295358]], dtype=float32)>

**Division**


In [43]:
a / b

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.13100992, -0.4141641 ],
       [ 0.01860114,  0.25213224],
       [-1.2261662 , -0.00263845]], dtype=float32)>

In [44]:
tf.divide(a, b)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.13100992, -0.4141641 ],
       [ 0.01860114,  0.25213224],
       [-1.2261662 , -0.00263845]], dtype=float32)>

**Module reminder**


In [45]:
a % b

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[5.502992 , 2.5837674],
       [0.1251936, 1.1770082],
       [0.7381774, 3.5247617]], dtype=float32)>

In [46]:
tf.math.floormod(a, b)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[5.502992 , 2.5837674],
       [0.1251936, 1.1770082],
       [0.7381774, 3.5247617]], dtype=float32)>

**Matrix Multiplication**

In [47]:
a @ c

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[10.669503  ,  0.16929226, -6.425128  ],
       [-9.005304  , -0.09658577,  5.115662  ],
       [-6.009898  ,  0.03644732,  2.7443535 ]], dtype=float32)>

In [48]:
tf.matmul(a, c)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[10.669503  ,  0.16929226, -6.425128  ],
       [-9.005304  , -0.09658577,  5.115662  ],
       [-6.009898  ,  0.03644732,  2.7443535 ]], dtype=float32)>

**Floor Division**

In [49]:
a // b

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-1., -1.],
       [ 0.,  0.],
       [-2., -1.]], dtype=float32)>

**Exponent**

In [50]:
a ** 3

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-5.7103759e-01, -6.0946631e+00],
       [ 1.9622138e-03,  1.6305661e+00],
       [-1.6002483e+00, -8.1073011e-07]], dtype=float32)>

In [51]:
tf.pow(a, 3)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-5.7103759e-01, -6.0946631e+00],
       [ 1.9622138e-03,  1.6305661e+00],
       [-1.6002483e+00, -8.1073011e-07]], dtype=float32)>

<a id='Comparison'></a>
### 4.2 Comparison Operators 

Comparison operators are used to compare tensors.


In [52]:
a.numpy()

array([[-0.8296372, -1.8266271],
       [ 0.1251936,  1.1770082],
       [-1.1696676, -0.0093245]], dtype=float32)

In [53]:
# Greater than

a > 1

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[False, False],
       [False,  True],
       [False, False]])>

In [54]:
# Greater than or equal to
tf.abs(a) >= 1

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[False,  True],
       [False,  True],
       [ True, False]])>

In [55]:
# Less than

a < 1

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True, False],
       [ True,  True]])>

In [56]:
# Not equal to

a != 1

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True,  True],
       [ True,  True]])>

In [57]:
# Equal to

a == 1


<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[False, False],
       [False, False],
       [False, False]])>

<a id='Logical'></a>
### 4.3 Logical And Bitwise Operators

Logical operators (`AND`, `OR`, `NOT`) are used to compare expressions.

In [58]:
tf.abs(a) > 1

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[False,  True],
       [False,  True],
       [ True, False]])>

In [59]:
b <= 3.5

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[False, False],
       [False, False],
       [ True, False]])>

`AND` returns `True` if both conditions are `True`.

In [60]:
True and True

True

In [61]:
True and False


False

To compare the expressions that involve tensors, we can use bitwise operators.

In [62]:
# AND

(tf.abs(a) > 1) & (b <= 3.5)

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[False, False],
       [False, False],
       [ True, False]])>

In [63]:
# NOT

~ ((tf.abs(a) > 1)  & (b <= 3.5))

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True,  True],
       [False,  True]])>

In [64]:
# OR

(tf.abs(a) > 1) | (b <= 3.5)

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[False,  True],
       [False,  True],
       [ True, False]])>

### [TOP ☝️](#top)